# Requirements

In [ ]:
%%capture
!pip uninstall -y transformers && pip install git+https://github.com/huggingface/transformers
!pip install datasets

# Loading model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-128k-instruct", attn_implementation="eager")
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.35k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Connect to Drive

In [1]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
%cd 'drive/MyDrive/Projects/Dastpokht/comment_analyzer/data'

Mounted at /content/drive
/content/drive/MyDrive/Projects/Dastpokht/comment_analyzer/data


# Loading Data

In [2]:
df = pd.read_csv("./analyzer_result.csv", encoding='utf-8')
print(f"Shape of my Data: {df.shape}\n")
df.head()

Shape of my Data: (5152, 4)



,comment_text,qualities,services,foods
0,برگر با کیفیت. تحویل سریع. گرم هم رسید غذا.,"{""{\""food_rate\"": \""5\"", \""food_title\"": \""سان...","{""{\""service_rate\"": \""5\"", \""part_of_comment\...","{""{\""product_id\"": 18749645, \""product_title\""..."
1,عالي بود,"{""{\""food_rate\"": \""5\"", \""food_title\"": \""است...","{""{\""service_rate\"": \""Nan\"", \""part_of_commen...","{""{\""product_id\"": 18748885, \""product_title\""..."
2,خیلی خوب بود ممنون,"{""{\""food_rate\"": \""5\"", \""food_title\"": \""آمر...","{""{\""service_rate\"": \""Nan\"", \""part_of_commen...","{""{\""product_id\"": 18748883, \""product_title\""..."
3,کیفیت برگرا عالی ولی سیب زمینی خیلی معمولی,"{""{\""food_rate\"": \""Nan\"", \""food_title\"": \""ن...","{""{\""service_rate\"": \""Nan\"", \""part_of_commen...","{""{\""product_id\"": 18749778, \""product_title\""..."
4,خیلی خوب بود,"{""{\""food_rate\"": \""5\"", \""food_title\"": \""آمر...","{""{\""service_rate\"": \""Nan\"", \""part_of_commen...","{""{\""product_id\"": 18748883, \""product_title\""..."


# Preprocessing Data

In [3]:
import json
import re
import ast

comment_text_array = df['comment_text'].to_numpy()
foods_text_array = df['foods'].to_numpy()
qualities_text_array = df['qualities'].to_numpy()
services_text_array = df['services'].to_numpy()

primary_inputs = []
primary_results = []

for comment, food, qualities, services in zip(comment_text_array, foods_text_array, qualities_text_array, services_text_array):
    json_food = ast.literal_eval(food)
    json_qualities = ast.literal_eval(qualities)
    json_services = ast.literal_eval(services)
    # Print the converted dictionary

    current_title = ""
    current_materials = ""

    for product , quality in zip(json_food, json_qualities):
        json_product = ast.literal_eval(product)
        json_quality = ast.literal_eval(quality)
        current_materials = current_materials +" , "+ "{"+ json_product['product_title']+ " : " +json_product['product_description']+"}"
        current_title = current_title+" , "+ " [ "+ json_product['product_title'] + " ] "

    primary_inputs.append({
            "comment": comment,
            "materials": current_materials,
            "foods": current_title
        })

    primary_results.append({
            "comment_text": comment,
            "order": current_title,
            "response": " [ "+ str(json_qualities) + " ] " +" , "+ " [ "+  str(json_services) + " ] "
        })

# Saving Prepared Data

In [ ]:
import csv

def write_data_to_csv(data, filename, fieldnames):

    # Open the file for writing
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        # Create a writer object specifying the field names
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Write the header row
        writer.writeheader()

        # Write data rows
        for row in data:
            writer.writerow(row)

# Call the function with your data and desired file name
write_data_to_csv(primary_inputs, 'primary_inputs.csv', ['comment', 'materials', 'foods'])
write_data_to_csv(primary_results, 'primary_results.csv', ['comment_text', 'order', 'response'])

a = pd.read_csv("./primary_inputs.csv", encoding='utf-8')
b = pd.read_csv("./primary_results.csv", encoding='utf-8')

a.head()
print(b.loc[0, 'response'])

 [ {'{"food_rate": "5", "food_title": "سیمپل فرایز"}', '{"food_rate": "5", "food_title": "ساندویچ رست بیف چدار لارج"}'} ]  ,  [ {'{"service_rate": "5", "part_of_comment": "تحویل سریع", "service_classified": "Delivery time"}', '{"service_rate": "5", "part_of_comment": "گرم هم رسید غذا", "service_classified": "Food temperature"}'} ] 


In [ ]:
prompt_enhancements = """Suppose that we are working in the domain of a food delivery app.
Here is the list of foods and beverages ordered by a user, let's call it 'UserOrder':
UserOrder: {order}

After receiving the 'UserOrder', here is the user's comment (review) about their experience having UserOrder (called 'UserComment'):
UserComment: {comment}
As you see, UserComment expresses the user's satisfaction with the food quality and services delivered.

If available, here is the list of ingredients per each food included in the UserOrder (called 'FoodIngredients'):
FoodIngredients: {ingredients}

Having the context provided, your task is to evaluate the food quality and services delivered based on UserComment. In other words, you should assign a score per each food and services delivered. Let's think step by step:
First, inspect the UserComment precisely.

Then, Based on UserComment, assign a score per each of the foods ordered (an integer between 0 and 5). In other words, all the food titles mentioned in the UserOrder must get a score field. If UserComment does not include any specific review for any of the food ordered, set the corresponding food score to 'Nan'.
*ATTENTION*: In this step, if UserComment is a general positive or negative comment and does not specify any particular food item, apply it to all food items ordered.
For example: Having UserComment='همه چی عالی', all the food items ordered will get a 5 score.

For the next step, refer to UserComment again. Based on UserComment, assign a score per each service provided (if any). To achieve this, first, you need to specify the relevant part of the UserComment expressing the user's experience with the service. Then, classify that service into one of these categories:
'[Delivery time, Quality of packaging, Cost of packaging, Cost per value of foods, Overall quality, Staff behavior, Food temperature, Delivered food not as ordered]'
After that, assign a score (an integer between 0 and 5) to that service based on part_of_comment. Feel free to set the service score as 'Nan' if the user seems unclear about the satisfaction with the service provided.

*ATTENTION*: As you see, the UserComment is in Persian. Thus, be mindful of any different Persian idioms or slang used and their actual meanings.

Finally, Based on the information inferred in the previous steps, generate JSON about the foods and services in this format:
```
{{
    qualities: [
        {{
            food_title: <food_title>,
            food_rate: <food_rate>
        }}
    ],
    services: [
        {{
            part_of_comment: <part_of_comment>,
            service_rate: <service_rate>,
            service_classified: <service_classified>
        }}
    ]
}}
```
Output only the JSON. Do not output any extra information or explanations.

"""

prompt_enhancements2 = """Suppose that we are working in the domain of a food delivery app.
Here is the list of foods and beverages ordered by a user, let's call it 'UserOrder':
UserOrder: {order}

After receiving the 'UserOrder', here is the user's comment (review) about their experience with UserOrder (called 'UserComment'):
UserComment: {comment}
As you see, UserComment expresses the user's satisfaction with the food quality and services delivered.

If available, here is the list of ingredients for each food included in the UserOrder (called 'FoodIngredients'):
FoodIngredients: {ingredients}


Having the context provided, your task is to evaluate the food quality and services delivered based on UserComment, and assign a score per each food and services delivered (an integer between 0 and 5) considering the following rules:

1. Rate Food:
   All the food titles mentioned in the UserOrder, must each receive a score (an integer between 0 and 5), considering the 'Handle Mixed Sentiments' rule (rule number 4). First, decide whether the UserComment is anything related to user's satisfaction about the quality of any food item in UserOrder.
   For this purpose, check if the UserComment directly shares sentiments about specific foods or food ingredients. If it contains sentiments about the food, then rate that food based on the UserComment. If it contains sentiments about the food ingredients,
   then check the food or foods containing those ingredients, utilizing the FoodIngredients list that contains each food with its ingredients and rate that food based on the UserComment.
   if the user has commented the overall food experience and the comment does not specify which food item the rating is for, you can use UserComment to rate all the food items ordered.
   If UserComment does not include any specific review for a specific food item of the UserOrder, set the corresponding food score to 'Nan'.

2. Separate Service Quality Comments:
   The UserComment may contain sentiments about each of the foods or the services, or both. First, decide whether the UserComment is anything related to these predefined service_classified categorie: '[Delivery time, Quality of packaging, Cost of packaging, Cost per value of foods, Overall quality, Staff behavior, Food temperature, Delivered food not as ordered]'.
   If the UserComment is related to one or more of these service_classified categorie, then extract and analyze parts of UserComment that pertain specifically to those service_classified categorie (separating them from comments about food quality), and attribute those extracted parts of the comment to part_of_comment.
   After that, assign a score (an integer between 0 and 5) to that service based on part_of_comment.
   If a service-related UserComment doesn’t fit any category or if the sentiment about the service is unclear, set 'part_of_comment', 'service_rate', and 'service_classified' to 'Nan'. Note that there might be more than one service mentioned in the comment.

3. Evaluate Food Temperature:
   If UserComment mentions the temperature of the food, identify which food is being discussed and determine whether it is typically served hot or cold.
   For example, fast foods and main dishes are often served hot, while most drinks and salads are served cold.
   Then, evaluate if the UserComment indicates a satisfactory temperature. For example, the model should asign a low rate for a dish that is mentioned to be 'cold' in the UserComment, but it is often served hot.
   But id the meal is often served 'cold' and it is mentioned to be 'cold' in the UserComment, the meal should gets a high rate.

4. Handle Mixed Sentiments:
   If UserComment contains mixed sentiments (e.g., positive followed by negative), consider the overall tone of the comment at its conclusion. Focus on how the UserComment ends to determine if it’s generally positive or negative. This prevents confusion over mixed messages within the comment.

*ATTENTION1*: As you see, the UserComment is in Persian. Thus, be mindful of any different Persian idioms or slang used and their actual meanings.
*ATTENTION2*: In this step, if UserComment is a general positive or negative UserComment and does not specify any particular food item, apply it to all food items ordered.
For example: Having UserComment='همه چی عالی', all the food items ordered will get a 5 score.


Based on the analysis, generate JSON for foods and services in this format:
 {{
    qualities: [
        {{
            food_title: <food_title>,
            food_rate: <food_rate>
        }}
    ],
    services: [
        {{
            part_of_comment: <part_of_comment>,
            service_rate: <service_rate>,
            service_classified: <service_classified>
        }}
    ]
}}
```
Output only the JSON. Do not output any extra information or explanations."""


system_role = "You are a helpful digital assisstant having brilliant analytical skills."

In [ ]:
print(prompt_enhancements2)

Suppose that we are working in the domain of a food delivery app.
Here is the list of foods and beverages ordered by a user, let's call it 'UserOrder':
UserOrder: {order}

After receiving the 'UserOrder', here is the user's comment (review) about their experience with UserOrder (called 'UserComment'):
UserComment: {comment}
As you see, UserComment expresses the user's satisfaction with the food quality and services delivered.

If available, here is the list of ingredients for each food included in the UserOrder (called 'FoodIngredients'):
FoodIngredients: {ingredients}


Having the context provided, your task is to evaluate the food quality and services delivered based on UserComment, and assign a score per each food and services delivered (an integer between 0 and 5) considering the following rules:

1. Rate Food:
   All the food titles mentioned in the UserOrder, must each receive a score (an integer between 0 and 5), considering the 'Handle Mixed Sentiments' rule (rule number 4). Fi

In [ ]:
result = pd.read_csv("./primary_results.csv", encoding='utf-8')
result.head()

,comment_text,order,response
0,برگر با کیفیت. تحویل سریع. گرم هم رسید غذا.,", [ سیمپل فرایز ] , [ ساندویچ رست بیف چدار...","[ {'{""food_rate"": ""5"", ""food_title"": ""سیمپل ف..."
1,عالي بود,", [ نوشابه قوطی زیرو ] , [ استیک برگر ]","[ {'{""food_rate"": ""5"", ""food_title"": ""نوشابه ..."
2,خیلی خوب بود ممنون,", [ آمریکانا برگر ]","[ {'{""food_rate"": ""5"", ""food_title"": ""آمریکان..."
3,کیفیت برگرا عالی ولی سیب زمینی خیلی معمولی,", [ ساندویچ رست چیکن ] , [ جوسی برگر ] , ...","[ {'{""food_rate"": ""Nan"", ""food_title"": ""ساندو..."
4,خیلی خوب بود,", [ آمریکانا برگر ] , [ سیمپل فرایز ] , ...","[ {'{""food_rate"": ""5"", ""food_title"": ""آمریکان..."


# Generating Response

In [ ]:
for index, row in enumerate(primary_inputs[30:31]):
    prompt = prompt_enhancements2.format(order=row['foods'], comment=row['comment'], ingredients=row['materials'])
    messages = [{"role": "system",
            "content": f"{system_role}"},
    {"role": "user",
    "content": f"{prompt}"}]


    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")

    outputs = model.generate(inputs, max_new_tokens=2000)
    text = tokenizer.batch_decode(outputs[:, inputs.shape[1]:])[0]
    print(messages)
    print("-"*50)
    print(text)
    result.loc[index, "response"] = text
    print(f"{index} Done.")
    print("*"*50)

[{'role': 'system', 'content': 'You are a helpful digital assisstant having brilliant analytical skills.'}, {'role': 'user', 'content': "Suppose that we are working in the domain of a food delivery app.\nHere is the list of foods and beverages ordered by a user, let's call it 'UserOrder':\nUserOrder:  ,  [ ساندویچ زینگر نرمال ]  ,  [ برگر کلاسیک ]  ,  [ شات پنیر دیپ چدار ]  ,  [ سیب زمینی سرخ کرده ] \n\nAfter receiving the 'UserOrder', here is the user's comment (review) about their experience with UserOrder (called 'UserComment'):\nUserComment: دفعه سوم با کلی ذوق سفارش دادم ولی راضی نبودم واقعا\nAs you see, UserComment expresses the user's satisfaction with the food quality and services delivered.\n\nIf available, here is the list of ingredients for each food included in the UserOrder (called 'FoodIngredients'):\nFoodIngredients:  , {ساندویچ زینگر نرمال : (150 گرم) یک عدد سینه مرغ سوخاری، کاهو، گوجه، خیار شور سس مخصوص و پنیر گودا} , {برگر کلاسیک : نان برگر ، 140 گرم برگر گوشت 90 درصد

# Evaluate Results

In [ ]:
for i in range(30,33):
    print("comment text: ")
    print(result["comment_text"][i])
    print("Order: ")
    print(result["order"][i])
    print("Model response: ")
    print(result.loc[i, "response"])
    print("*"*50)

comment text: 
دفعه سوم با کلی ذوق سفارش دادم ولی راضی نبودم واقعا
Order: 
 ,  [ برگر کلاسیک ]  ,  [ سیب زمینی سرخ کرده ]  ,  [ شات پنیر دیپ چدار ]  ,  [ ساندویچ زینگر نرمال ] 
Model response: 
{'{"food_rate": "Nan", "food_title": "ساندویچ زینگر نرمال"}', '{"food_rate": "Nan", "food_title": "سیب زمینی سرخ کرده"}', '{"food_rate": "Nan", "food_title": "برگر کلاسیک"}', '{"food_rate": "Nan", "food_title": "شات پنیر دیپ چدار"}'}
**************************************************
comment text: 
با کیفیت بود ممنونم
Order: 
 ,  [ سیب زمینی سرخ کرده ] 
Model response: 
{'{"food_rate": "5", "food_title": "سیب زمینی سرخ کرده"}'}
**************************************************
comment text: 
سرد بود
Order: 
 ,  [ فیله استریپس سوخاری ]  ,  [ پیتزا چیکن آلفردو ] 
Model response: 
{'{"food_rate": "Nan", "food_title": "فیله استریپس سوخاری"}', '{"food_rate": "Nan", "food_title": "پیتزا چیکن آلفردو"}'}
**************************************************


# Test with original prompts

In [ ]:
original_prompt = """Suppose that we are working in the domain of a food delivery app.
Here is the list of foods and beverages ordered by a user, let's call it 'UserOrder':
UserOrder: {order}

After receiving the 'UserOrder', here is the user's comment (review) about their experience having UserOrder (called 'UserComment'):
UserComment: {comment}
As you see, UserComment expresses the user's satisfaction with the food quality and services delivered.

If available, here is the list of ingredients per each food included in the UserOrder (called 'FoodIngredients'):
FoodIngredients: {ingredients}

Having the context provided, your task is to evaluate the food quality and services delivered based on UserComment. In other words, you should assign a score per each food and services delivered. Let's think step by step:
First, inspect the UserComment precisely.

Then, Based on UserComment, assign a score per each of the foods ordered (an integer between 0 and 5). In other words, all the food titles mentioned in the UserOrder must get a score field. If UserComment does not include any specific review for any of the food ordered, set the corresponding food score to 'Nan'.
*ATTENTION*: In this step, if UserComment is a general positive or negative comment and does not specify any particular food item, apply it to all food items ordered.
For example: Having UserComment='همه چی عالی', all the food items ordered will get a 5 score.

For the next step, refer to UserComment again. Based on UserComment, assign a score per each service provided (if any). To achieve this, first, you need to specify the relevant part of the UserComment expressing the user's experience with the service. Then, classify that service into one of these categories:
'[Delivery time, Quality of packaging, Cost of packaging, Cost per value of foods, Overall quality, Staff behavior, Food temperature, Delivered food not as ordered]'
After that, assign a score (an integer between 0 and 5) to that service based on part_of_comment. Feel free to set the service score as 'Nan' if the user seems unclear about the satisfaction with the service provided.

*ATTENTION*: As you see, the UserComment is in Persian. Thus, be mindful of any different Persian idioms or slang used and their actual meanings.

Finally, Based on the information inferred in the previous steps, generate JSON about the foods and services in this format:
{{
    qualities: [
        {{
            "food_title": "",
            "food_rate": ""
        }}
    ],
    services: [
        {{
            part_of_comment: "",
            service_rate: "",
            service_classified: ""
        }}
    ]
}}
Output only the JSON. Do not output any extra information or explanations.

"""


In [ ]:
result3= pd.read_csv("./primary_results.csv", encoding='utf-8')
result3.head()

,comment_text,order,response
0,برگر با کیفیت. تحویل سریع. گرم هم رسید غذا.,", [ ساندویچ رست بیف چدار لارج ] , [ سیمپل ...","{'{""food_rate"": ""5"", ""food_title"": ""سیمپل فرای..."
1,عالي بود,", [ استیک برگر ] , [ نوشابه قوطی زیرو ]","{'{""food_rate"": ""5"", ""food_title"": ""استیک برگر..."
2,خیلی خوب بود ممنون,", [ آمریکانا برگر ]","{'{""food_rate"": ""5"", ""food_title"": ""آمریکانا ب..."
3,کیفیت برگرا عالی ولی سیب زمینی خیلی معمولی,", [ ساندویچ رست چیکن ] , [ جوسی برگر ] , ...","{'{""food_rate"": ""Nan"", ""food_title"": ""نوشابه ق..."
4,خیلی خوب بود,", [ ساندویچ رست چیکن ] , [ آمریکانا برگر ]...","{'{""food_rate"": ""5"", ""food_title"": ""سیمپل فرای..."


In [ ]:
for index, row in enumerate(primary_inputs[30:42]):
    prompt = original_prompt.format(order=row['foods'], comment=row['comment'], ingredients=row['materials'])
    messages = [{"role": "system",
            "content": f"{system_role}"},
    {"role": "user",
    "content": f"{prompt}"}]


    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")

    outputs = model.generate(inputs, max_new_tokens=2000, temperature=0.0)
    text = tokenizer.batch_decode(outputs[:, inputs.shape[1]:])[0]
    print(messages)
    print("-"*50)
    print(text)
    result3.loc[index, "response"] = text
    print(f"{index} Done.")
    print("*"*50)


0 Done.
1 Done.
2 Done.
3 Done.
4 Done.
5 Done.
6 Done.
7 Done.
8 Done.
9 Done.
10 Done.
11 Done.


In [ ]:
for i in range(30,43):
    print("comment text: ")
    print(result3["comment_text"][i])
    print("Order: ")
    print(result3["order"][i])
    print("Model response: ")
    print(result3["response"][i])
    print("*"*50)

comment text: 
دفعه سوم با کلی ذوق سفارش دادم ولی راضی نبودم واقعا
Order: 
 ,  [ برگر کلاسیک ]  ,  [ ساندویچ زینگر نرمال ]  ,  [ شات پنیر دیپ چدار ]  ,  [ سیب زمینی سرخ کرده ] 
Model response: 
{'{"food_rate": "Nan", "food_title": "سیب زمینی سرخ کرده"}', '{"food_rate": "Nan", "food_title": "برگر کلاسیک"}', '{"food_rate": "Nan", "food_title": "ساندویچ زینگر نرمال"}', '{"food_rate": "Nan", "food_title": "شات پنیر دیپ چدار"}'}
**************************************************
comment text: 
با کیفیت بود ممنونم
Order: 
 ,  [ سیب زمینی سرخ کرده ] 
Model response: 
{'{"food_rate": "5", "food_title": "سیب زمینی سرخ کرده"}'}
**************************************************
comment text: 
سرد بود
Order: 
 ,  [ فیله استریپس سوخاری ]  ,  [ پیتزا چیکن آلفردو ] 
Model response: 
{'{"food_rate": "Nan", "food_title": "پیتزا چیکن آلفردو"}', '{"food_rate": "Nan", "food_title": "فیله استریپس سوخاری"}'}
**************************************************
comment text: 
حجم غذا بسیار کم بود. اصلا مثل ۳ت

In [ ]:
result3.head(200)

,comment_text,order,response
0,برگر با کیفیت. تحویل سریع. گرم هم رسید غذا.,", [ ساندویچ رست بیف چدار لارج ] , [ سیمپل ...",<s><|user|> Suppose that we are working in the...
1,عالي بود,", [ استیک برگر ] , [ نوشابه قوطی زیرو ]",<s><|user|> Suppose that we are working in the...
2,خیلی خوب بود ممنون,", [ آمریکانا برگر ]",<s><|user|> Suppose that we are working in the...
3,کیفیت برگرا عالی ولی سیب زمینی خیلی معمولی,", [ ساندویچ رست چیکن ] , [ جوسی برگر ] , ...",<s><|user|> Suppose that we are working in the...
4,خیلی خوب بود,", [ ساندویچ رست چیکن ] , [ آمریکانا برگر ]...",<s><|user|> Suppose that we are working in the...
...,...,...,...
195,بهترینید,", [ همبرگر ]","{'{""food_rate"": ""5"", ""food_title"": ""همبرگر""}'}"
196,از همه لحاظ عالی,", [ کمبو یامی ]","{'{""food_rate"": ""5"", ""food_title"": ""کمبو یامی""}'}"
197,خوب بود,", [ فرش چیز برگر ]","{'{""food_rate"": ""5"", ""food_title"": ""فرش چیز بر..."
198,عالی بود,", [ همبرگر ]","{'{""food_rate"": ""5"", ""food_title"": ""همبرگر""}'}"


Testing with Main prompt

In [ ]:
main_prompt = """Your task is three things that i mentioned below:
    1_ For each food item, you need to:
        - Assign a rating based on quality of food(an integer between 0 and 5)
    2_ For a comment, you need to:
        - Extract the part of the comment that explain about quality of service.(part_of_comment)
        - Classify service based on this qualities: Delivery time, Quality of packaging, Cost of packaging, Cost per value of foods, Overall quality, Staff behavior, Food temperature, Delivered food not as ordered
        - Assign a rating based on quality of service(an integer between 0 and 5)
        - If the comment was not in service classifier fill part_of_comment, service_rate, service_classified with 'Nan'.
        - The comment might have more than one service.
    3_ Your output must be sorted look like below. Don't use extra information and explanation.
                {{
    qualities: [
        {{
            food_title: 'food_title',
            food_rate: 'food_rate'
        }}
    ],
    services: [
        {{
            part_of_comment: 'part_of_comment',
            service_rate: 'service_rate',
            service_classified: 'service_classified'
        }}
    ]
}}
    The Data is below within the three backticks.
    ```
    comment : {comment}
    materials : {ingredients}

    foods : {order}
    ```
    There is delivery service and customers order one or many different foods and after receiving their food, they can comment on each food item.
    The food items are named in Persian and listed as follows:: ['name of food','name of food', ...].
    The menu of the restaurant and the ingredients of each food item are sorted as follows:
      [{{'foods name': 'materials'}}, {{'foods name': 'materials'}}, ... ]
    pay attention to materials of foods. customer might comment and criticising about materials of food. foods might doesn't have materials.
    analyze the comment and determine which food item it is about.
    if the customer has commented the overall food experience and the comment does not specify which food item the rating is for, you can use comment for all the food items ordered.
    if the customer has commented some foods poorly and has not mentioned other foods or the overall experience, you must fill rate the other foods with 'Nan'.
    if the customer has commented poorly on some food items and has not mentioned other food items or the overall experience, you must not rate on other food items and fill it in with 'Nan'
    If the customer commented badly on some foods and other parts of the comment were the overall experience, use part of the comment with the overall experience on the other foods. The rate for bad foods must be exactly the same part of the sentences criticizing the foods.
    Pay attention to this two comment. they are rated as 5 for quality of foods: 'خدایییش غذاش خوبه', 'حرف نداره'
"""

In [ ]:
result4= pd.read_csv("./primary_results.csv", encoding='utf-8')
result4.head()

,comment_text,order,response
0,برگر با کیفیت. تحویل سریع. گرم هم رسید غذا.,", [ ساندویچ رست بیف چدار لارج ] , [ سیمپل ...","{'{""food_rate"": ""5"", ""food_title"": ""سیمپل فرای..."
1,عالي بود,", [ استیک برگر ] , [ نوشابه قوطی زیرو ]","{'{""food_rate"": ""5"", ""food_title"": ""استیک برگر..."
2,خیلی خوب بود ممنون,", [ آمریکانا برگر ]","{'{""food_rate"": ""5"", ""food_title"": ""آمریکانا ب..."
3,کیفیت برگرا عالی ولی سیب زمینی خیلی معمولی,", [ ساندویچ رست چیکن ] , [ جوسی برگر ] , ...","{'{""food_rate"": ""Nan"", ""food_title"": ""نوشابه ق..."
4,خیلی خوب بود,", [ ساندویچ رست چیکن ] , [ آمریکانا برگر ]...","{'{""food_rate"": ""5"", ""food_title"": ""سیمپل فرای..."


In [ ]:
for index, row in enumerate(primary_inputs[30:42]):
    prompt = original_prompt.format(order=row['foods'], comment=row['comment'], ingredients=row['materials'])
    messages = [{"role": "system",
            "content": f"{system_role}"},
    {"role": "user",
    "content": f"{prompt}"}]


    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")

    outputs = model.generate(inputs, max_new_tokens=2000, temperature=0.0)
    text = tokenizer.batch_decode(outputs[:, inputs.shape[1]:])[0]
    print(messages)
    print("-"*50)
    print(text)
    result4.loc[index, "response"] = text
    print(f"{index} Done.")
    print("*"*50)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:490: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


0 Done.
1 Done.
2 Done.
3 Done.
4 Done.
5 Done.
6 Done.
7 Done.
8 Done.
9 Done.
10 Done.
11 Done.


In [ ]:
for i in range(30,43):
    print("Comment text: ")
    print(result4["comment_text"][i])
    print("Order: ")
    print(result4["order"][i])
    print("Model response: ")
    print(result4["response"][i])
    print("*"*50)

Comment text: 
دفعه سوم با کلی ذوق سفارش دادم ولی راضی نبودم واقعا
Order: 
 ,  [ برگر کلاسیک ]  ,  [ ساندویچ زینگر نرمال ]  ,  [ شات پنیر دیپ چدار ]  ,  [ سیب زمینی سرخ کرده ] 
Model response: 
{'{"food_rate": "Nan", "food_title": "سیب زمینی سرخ کرده"}', '{"food_rate": "Nan", "food_title": "برگر کلاسیک"}', '{"food_rate": "Nan", "food_title": "ساندویچ زینگر نرمال"}', '{"food_rate": "Nan", "food_title": "شات پنیر دیپ چدار"}'}
**************************************************
Comment text: 
با کیفیت بود ممنونم
Order: 
 ,  [ سیب زمینی سرخ کرده ] 
Model response: 
{'{"food_rate": "5", "food_title": "سیب زمینی سرخ کرده"}'}
**************************************************
Comment text: 
سرد بود
Order: 
 ,  [ فیله استریپس سوخاری ]  ,  [ پیتزا چیکن آلفردو ] 
Model response: 
{'{"food_rate": "Nan", "food_title": "پیتزا چیکن آلفردو"}', '{"food_rate": "Nan", "food_title": "فیله استریپس سوخاری"}'}
**************************************************
Comment text: 
حجم غذا بسیار کم بود. اصلا مثل ۳ت

In [ ]:

df1= pd.read_json("500.json", encoding="utf-8")
df1.head()

,comment,materials,foods
0,سیب زمینی نداشت وکم بود,[{'ساندویچ ران بریان': '300 گرم ران مرغ بریان،...,[ساندویچ ران بریان]
1,نپخته و خام ارسال شد وقتی غذا حاضر نیست چرا ار...,[{'مرغ بریان کامل': 'یک عدد مرغ بریان کامل 1.3...,[مرغ بریان کامل]
2,افتضاح مرغ و سیب زمینی بو میداد نون هم برام نف...,[{'ران مرغ بریان': 'یک عدد ران مرغ بریان شده 4...,"[ران مرغ بریان, نان باگت, سیب زمینی تنوری]"
3,یک چیز دیگه برام ارسال کردن,[{'مرغ بریان کامل': 'یک عدد مرغ بریان کامل 1.3...,"[مرغ بریان کامل, نوشابه خانواده کوکا کولا]"
4,واقعا بد بود,[{'کتف مرغ بریان': '400 گرم کتف مرغ بریان شده،...,"[کتف مرغ بریان, بال مرغ بریان, مرغ بریان کامل]"
